                                  Long end Short


Estudo de long end short, para ativos que compõem a carteira B3 que tem ticker BOVA11.

O importante de ser esta carteira, trabalhar com opções (Derivativos), assim diminuindo risco da estratégia.

Foi usado dados MetaTrade5 pois são os que menos apresenta dados nulos e dados faltantes.
 
Este trabalho foi feito em cima da forma clássica:
- Vários períodos de tempo 
- Cointegração 
- Dickey-Fuller
- Resíduos 
Conta com gráfico de fácil visualização 


In [ ]:
# importando as bibliotecas 

import datetime as dt
import time
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
from pandas.io.html import read_html
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# URL da página da Wikipedia
page = 'https://pt.wikipedia.org/wiki/Lista_de_companhias_citadas_no_Ibovespa'

# Lê a tabela da página usando pandas
table = pd.read_html(page, attrs={'class': 'wikitable'})

# Obtém a coluna 'Código' e adiciona '.SA' a cada valor
dft = table[0]['Código']
#dft = [i + '.SA' for i in dft]

print(dft)

In [ ]:
# Inicializa a conexão com o terminal MetaTrader 5
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())

# Cria um DataFrame vazio
df = pd.DataFrame()

# Coleta dados de fechamento para cada ticker na lista
for ticker in dft:
    rates = mt5.copy_rates_from_pos(ticker, mt5.TIMEFRAME_D1, 0, 250)
    dfa = pd.DataFrame(rates)
    
    
    if 'close' in dfa.columns:
        df[ticker] = dfa['close']
    else:
        print(f"Este ticker '{ticker}' não foi localizado.")

# Coleta dados de fechamento para o ticker 'ITSA4' para parametros
rates = mt5.copy_rates_from_pos('ITSA4', mt5.TIMEFRAME_D1, 0, 250)
dfa = pd.DataFrame(rates)
dfa['time'] = pd.to_datetime(dfa['time'], unit='s')
df.set_index(dfa['time'], inplace=True)

# Exibe as primeiras linhas do DataFrame
print(df.head())

# Encerra a conexão com o MetaTrader 5
mt5.shutdown()

In [ ]:
# não irei usar yahoo finance pois tem muitos dados nulos 

# URL da página da Wikipedia
#page = 'https://pt.wikipedia.org/wiki/Lista_de_companhias_citadas_no_Ibovespa'

# Lê a tabela da página usando pandas
#table = pd.read_html(page, attrs={'class': 'wikitable'})

# Obtém a coluna 'Código' e adiciona '.SA' a cada valor
#dft = table[0]['Código']
#dft = [i + '.SA' for i in dft]

# Criando um DataFrame vazio para armazenar os preços de fechamento ajustados
#adj_close_df = pd.DataFrame()

# Iterando pelos tickers
#for ticker in dft:
    #try:
        # Obtendo os dados históricos para o ticker
        #data = yf.download(ticker, start='2020-01-01', end='2024-03-04')
        # Selecionando apenas a coluna 'Adj Close'
        #adj_close = data['Adj Close']
        # Adicionando os preços de fechamento ajustados ao DataFrame
       # adj_close_df[ticker] = adj_close
    #except Exception as e:
        #print(f"Erro ao obter dados para o ticker {ticker}: {e}")

# Exibindo o DataFrame com os preços de fechamento ajustados
#print(adj_close_df)



In [ ]:
# Excluindo colunas com valores nulos
#adj_close_df = df
#adj_close_df = adj_close_df.dropna(axis=1)

In [ ]:
df.info()

In [ ]:
# criando novas colunas com periodos de tempo 

df120 = df.iloc[-120:]
df140 = df.iloc[-140:]
df160 = df.iloc[-160:]
df180 = df.iloc[-180:]
df200 = df.iloc[-200:]
df220 = df.iloc[-220:]
df250 = df.iloc[-250:]
df.copy()
dfs = [df120,df140,df160,df180,df200,df220,df250]
for i in dfs:
    i.dropna(axis=1,inplace=True)

In [ ]:
# cointegração entre duas séries temporais (t1 e t2) utilizando o teste de ADF (Augmented Dickey-Fuller).

def cointegration(t1,t2,df):
    model = OLS(df[t1],df[t2])
    model = model.fit()
    adf = adfuller(model.resid)
    #print('test statistic : {}   ,  p-value : {} \n'.format(adf[0],adf[1]))
    #print('Criticals : {}'.format(adf[4]))
    if adf[1] < 0.05:
        return True
    else:
        return False

In [ ]:
# vamos aplicar cointegração em diferentes intervalos de tempo.

def get_pairs(): 
    
    pares= []
    dfs = [df120,df140,df160,df180,df200,df220,df250]
    for ticker1 in dfs[0].columns:
        for ticker2 in dfs[0].columns:
            aux=[]
            if ticker1 == ticker2:
                continue
            for timeframe in dfs:
                c = cointegration(ticker1,ticker2,timeframe)
                aux.append(c)
            #print(ticker1,ticker2,aux)
            if sum(aux) >= 3:
                pares.append('{}x{}'.format(ticker1,ticker2))
    return pares

In [ ]:
%%time
p = get_pairs()
print(p)

In [ ]:
# resíduos dos pares
 
def get_resid(par):
    t1 = df[par.split('x')[0]]
    t2 = df[par.split('x')[1]]
    model = OLS(t1,t2)
    model = model.fit()
    return model.resid

In [ ]:
#  cointegração entre os ativos e intervalo tempo

def cointegration_wtimes(ticker1,ticker2):
    l = [120,140,160,180,200,220,250]
    aux = {}
    for i in range(len(dfs)):
        co = cointegration(ticker1,ticker2,dfs[i])
        aux[l[i]] = co
    return aux


In [ ]:
# definindo limite superior e inferior 

def entradas(par):
    r = get_resid(par)
    upper_limit = 2 * r.std()
    lower_limit = -2 * r.std()
    if r.iloc[-1] > upper_limit or r.iloc[-1] < lower_limit:
        return par
    else:
        return False

# Crie uma lista para armazenar os pares de entrada válidos
lista_entradas = []
for par in p:
    a = entradas(par)
    if a:
        lista_entradas.append(a)

print(lista_entradas)

In [ ]:
# Crie a figura e os eixos
fig, ax = plt.subplots(len(lista_entradas), 1, figsize=(10, 20))
fig.subplots_adjust(hspace=0.5)

# sobre os pares de entrada
for i, par in enumerate(lista_entradas):
    r = get_resid(par)
    sns.lineplot(x=df.index, y=r, ax=ax[i])
    ax[i].axhline(y=2*r.std(), color='r', alpha=0.33)
    ax[i].axhline(y=-2*r.std(), color='r', alpha=0.33)
    ax[i].axhline(y=r.mean(), color='g', alpha=0.33)
    ax[i].set_ylabel("Spread")
    ax[i].set_xlabel("")
    ax[i].set_title(par)

plt.show()